# Test Model

In [1]:
import h5py
import model
import torch

n_hidden = 900
n_mixt_attention = 10
n_mixt_output = 20
batch_size = 50

char_dict = {k:v for v,k in enumerate(' !"#%&\'()+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]abcdefghijklmnopqrstuvwxyz')}
inv_char_dict = {v:k for k,v in char_dict.items()}
n_chars = len(char_dict)

def save_to_hdf5(fn, d):
    f = h5py.File(fn, "w")
    for k,v in d.items():
        f.create_dataset(k, data=v)

def load_from_hdf5(fn):
    f = h5py.File(fn, "r")
    print (list(f.keys()))
    return {k:v[:] for k,v in f.items()}

In [2]:
handwriting_gen = model.HandwritingModel(n_hidden, n_chars, n_mixt_attention, n_mixt_output)
print(handwriting_gen)

HandwritingModel(
  (attention): Attention(
    (linear): Linear(in_features=900, out_features=30, bias=True)
  )
  (rnn_cell): RNNCell(
    (cell): LSTMCell(84, 900)
  )
  (mixture): MixtureGaussians2DandPen(
    (linear): Linear(in_features=981, out_features=121, bias=True)
  )
)


In [3]:
weights = load_from_hdf5('pretrained_weights/graves_handwriting_generation_2018-03-13-02-45epoch_49.hd5')
for n,p in handwriting_gen.named_parameters():
        p.data.copy_(torch.from_numpy(weights[n]))

['attention.linear.bias', 'attention.linear.weight', 'mixture.linear.bias', 'mixture.linear.weight', 'rnn_cell.cell.bias_hh', 'rnn_cell.cell.bias_ih', 'rnn_cell.cell.weight_hh', 'rnn_cell.cell.weight_ih']


In [4]:
for bias in [0,2,2,2,5,5,5,100,100,100]:
    with torch.no_grad():
        i = next(iter(train_it))
        i_seq_pt = torch.stack([i.xs, i.ys, i.pen.float()], dim=2)
        i_seq_pt = i_seq_pt[:-1]

        i_seq_str = torch.autograd.Variable(torch.LongTensor([int(char_dict[c]) for c in "Die Katzen von Kopenhagen. "]).view(-1,1).cuda())
        i_seq_str_mask = torch.autograd.Variable(torch.ones(i_seq_str.size()).cuda())
        seq_pt, seq_mask = m.predict(i_seq_pt[0,:1], i_seq_str, i_seq_str_mask, bias=bias)
        lengths = seq_mask.sum(0).data.long()
        idx = 0
        pyplot.figure(figsize=(10,2))
        show_stroke(seq_pt.data.cpu()[:lengths[idx],idx])
        #''.join([inv_char_dict[c] for c in i_seq_str.data.cpu()[:,idx]])
        pyplot.title("bias={}".format(bias))
        pyplot.show()

NameError: name 'train_it' is not defined

In [11]:
i_seq_str = torch.autograd.Variable(torch.LongTensor([int(char_dict[c]) for c in "Hello Leo. "]).view(-1,1).cuda())
i_seq_str_mask = torch.autograd.Variable(torch.ones(i_seq_str.size()).cuda())
#seq_pt, seq_mask = handwriting_gen.predict(i_seq_pt[0,:1], i_seq_str, i_seq_str_mask, bias=bias)

NameError: name 'seq_mask' is not defined